## Providence Hospital Provider Affiliation Web Scrape

The goal of this project is to find all the provider info from the Providence FAP website (https://www.seton.net/providers/)

Notes: the complete list of provider info is on that page; each bio page contains affiliation info; Same website structure as Seton (both Ascension hosps)

### Overview
1. Get all provider info from initial page
2. Use provider Roster to get individual provider page list
3. Scrape affiliation information from individual provider page

### STEP 1: Get all Provider Info/Roster

In [2]:
import pandas as pd
import time
import numpy as np

In [3]:
import requests
page = requests.get("https://www.providence.net/providers/")
page.status_code

200

In [4]:
from bs4 import BeautifulSoup, CData
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
script = soup.findAll('script')

In [6]:
#all doc info in script[8]

#NOTE: the script has extra text; must be stripped
#start: <script type="text/javascript">
#//<![CDATA[
#RunGA = true;searchParams = {"SearchPhrase":"","GroupName":null,"SpecialtyName":null,"LastNameIs":null,"LastNameStartsWith":null,"IncludeMale":true,"IncludeFemale":true,"IncludeSFOD":false,"IncludeSHA":false,"IncludeC4T":false,"IncludeAccepting":false,"IncludePC":false,"LanguageName":null,"AffiliationName":null,"ProviderTypeName":null,"InsuranceName":null};defer(function() { jQuery(document).ready(function() {CheckMap(); }); });findADoctorResults = 
#end: ;//]]>
#</script>

import re
string = str(script[8])
stringlist1 = string.split('findADoctorResults = ')
stringlist1[0]
string2 = stringlist1[1]
stringlist2 = string2.split(';//]]')
stringlist2[1]
doclist = stringlist2[0]

In [7]:
#the remaining text is in JSON format; use pandas built-in function to convert to DF
fap_df = pd.read_json(doclist)

In [8]:
fap_df.head()

,AddressFormatted,Associations,Credentials,DisplayName,Distance,FirstName,FirstNameID,Groups,LastName,LastNameID,Latitude,Longitude,MapAddress,PhoneFormatted,Specialties
0,"405 Londonderry Dr <br />Suite 105 <br />Waco,...",C4T,MD,Chidi Achilefu,-1,Chidi,chidi,"Mid Tex Anesthesia Associates, P.A.",Achilefu,achilefu,31.513468,-97.19951,"405 Londonderry Dr Suite 105 Waco, TX 76712",254-776-0266,Anesthesiology
1,"7125 New Sanger Rd <br />Suite 516 <br />Waco,...",CE,MD,Oluwarotimi Adesina,-1,Oluwarotimi,oluwarotimi,Ascension Medical Group Providence Orthopedic ...,Adesina,adesina,31.515670,-97.20348,"7125 New Sanger Rd Suite 516 Waco, TX 76712",254-752-9638,Sports Medicine
2,3445 Executive Center Dr. <br />Suite 250 <br ...,,MD,Joan Admirand,-1,Joan,joan,Clinical Pathology Associates,Admirand,admirand,30.359690,-97.74781,"3445 Executive Center Dr. Suite 250 Austin, TX...",512-579-4000,Pathology
3,"3000 N. IH 35 <br />Suite 700 <br />Austin, TX...",,MD,Amin Al-Ahmad,-1,Amin,amin,Texas Cardiac Arrhythmia,Al-Ahmad,al_ahmad,30.227207,-97.74614,"3000 N. IH 35 Suite 700 Austin, TX 78705",512-807-3150,"Cardiology, Cardiac Electrophysiology"
4,"3000 North IH 35 <br />Suite 700 <br />Austin,...",,MD,Amin Al-Ahmad,-1,Amin,amin,Texas Cardiac Arrhythmia,Al-Ahmad,al_ahmad,30.227186,-97.74618,"3000 North IH 35 Suite 700 Austin, TX 78705",512-807-3150,"Cardiology, Cardiac Electrophysiology"


In [119]:
#check structure
print(fap_df.shape)
print(fap_df.columns)

(3385, 15)
Index(['AddressFormatted', 'Associations', 'Credentials', 'DisplayName',
       'Distance', 'FirstName', 'FirstNameID', 'Groups', 'LastName',
       'LastNameID', 'Latitude', 'Longitude', 'MapAddress', 'PhoneFormatted',
       'Specialties'],
      dtype='object')


#### Write out the Roster Scrape to xlsx

In [9]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('ProvidenceRoster_20190815.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
fap_df.to_excel(writer, sheet_name='RosterResults')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

#### Hospital Specialties

For this project, we are only interested in getting affiliation information for facility-based specialties: Anesthesiology, Radiology, Pathology, Neonatology, and Emergency Medicine. See also: any doc listed as a form of Hospitalist.

Each hospital website has slightly different specialties listed.  Here, I manually curate the list of specialties I want to scrape.

In [23]:
fap_df['Specialties'].value_counts()

Family Medicine                                                                  72
                                                                                 51
Emergency Medicine                                                               39
Pathology                                                                        28
Anesthesiology                                                                   20
Nephrology                                                                       13
Cardiology, Internal Medicine, Cardiac Electrophysiology                         12
Cardiology                                                                       11
Pulmonary Disease, Sleep Medicine, Critical Care Medicine, Pulmonary Medicine    11
Hospitalist                                                                      10
Obstetrics and Gynecology, Gynecology                                             8
Radiology                                                                   

In [24]:
#make list of hospital specialties to scrape
hosp_specs = ['Pathology','Anesthesiology','Hospitalist','Radiology','Radiology, Neuroradiology','Pediatric Emergency Medicine, Emergency Medicine','Emergency Medicine, Pediatrics','Pain Management, Anesthesiology','Hospitalist, Family Medicine']

In [25]:
#subset roster for all hosp spec docs
hospSpecFAP_df = fap_df[fap_df.Specialties.isin(hosp_specs)]
hospSpecFAP_df.head()

,AddressFormatted,Associations,Credentials,DisplayName,Distance,FirstName,FirstNameID,Groups,LastName,LastNameID,Latitude,Longitude,MapAddress,PhoneFormatted,Specialties
0,"405 Londonderry Dr <br />Suite 105 <br />Waco,...",C4T,MD,Chidi Achilefu,-1,Chidi,chidi,"Mid Tex Anesthesia Associates, P.A.",Achilefu,achilefu,31.513468,-97.199510,"405 Londonderry Dr Suite 105 Waco, TX 76712",254-776-0266,Anesthesiology
2,3445 Executive Center Dr. <br />Suite 250 <br ...,,MD,Joan Admirand,-1,Joan,joan,Clinical Pathology Associates,Admirand,admirand,30.359690,-97.747810,"3445 Executive Center Dr. Suite 250 Austin, TX...",512-579-4000,Pathology
35,"6901 Medical Parkway <br />Waco, TX 76712",,MD,Kathleen Berg,-1,Kathleen,kathleen,"US Acute Care Solutions, US Acute Care Solutio...",Berg,berg,31.514454,-97.199490,"6901 Medical Parkway Waco, TX 76712",254-751-4180,"Pediatric Emergency Medicine, Emergency Medicine"
36,6300 La Calma Dr. <br />Suite 200 <br />Austin...,,MD,Kathleen Berg,-1,Kathleen,kathleen,"US Acute Care Solutions, US Acute Care Solutio...",Berg,berg,30.324444,-97.703026,"6300 La Calma Dr. Suite 200 Austin, TX 78752",512-452-8533,"Pediatric Emergency Medicine, Emergency Medicine"
38,"6901 Medical Parkway <br />Waco, TX 76712",C4T,DO,John Biboa,-1,John,john,"McLennan County Hospitalist Services, P.C.",Biboa,biboa,31.514465,-97.199530,"6901 Medical Parkway Waco, TX 76712",254-751-4551,Hospitalist


In [ ]:
seven_day = soup.find(id="seven-day-forecast")
name_tags = seven_day.select(".fad-result-info .fad-result-name")
periods = [pt.get_text() for pt in period_tags]

### STEP 2: Get Affiliations from each doc's bio page

Once we have the hosp spec roster, I will use those doc names and individual provider websites to scrape affiliation data for each page.

In [33]:
#url structure "https://www.providence.net/providers/bio/kameron-harrison"

In [26]:
#testing example

#Chidi Achilefu
firstname = "Chidi"
lastname = "Achilefu"

url = "https://www.providence.net/providers/bio/" + firstname + "-" + lastname
docpage = requests.get(url)
print(docpage.status_code)
soup = BeautifulSoup(docpage.content, 'html5lib')
affiliations = soup.find(id="physicianAffiliations")
affil_tags = affiliations.select(".affiliation")
doc_affils = [at.get_text().rstrip().strip() for at in affil_tags]
doc_affils

200


['Providence Health Center']

In [29]:
#define web scrape function

def doc_affils(dataframe):
    affil_list = []
    seconds = 2 #define seconds to wait between get requests
    for index, row in dataframe.iterrows():
        firstname = row['FirstNameID']
        lastname = row['LastNameID']
        url = "https://www.providence.net/providers/bio/" + firstname + "-" + lastname #concat url
        docpage = requests.get(url,time.sleep(seconds)) #get html from url, with delay to prevent website blacklisting
        print(docpage.status_code)
        soup = BeautifulSoup(docpage.content, 'html5lib') #parse html with BeautifulSoup
        affiliations = soup.find(id="physicianAffiliations") #find physicianAffiliations section of website
        try: 
            affil_tags = affiliations.select(".affiliation") #get all affiliations via affiliation ids
            doc_affils1 = [at.get_text().rstrip().strip() for at in affil_tags] #get text and strip whitespace
            dict1 = {'firstName': firstname, 'lastName': lastname, 'hospitalAffiliations': doc_affils1} #make name and affils into dictionary
            affil_list.append(dict1)
            print("Added Affil for "+firstname+" "+lastname)
        except:
            pass
            print("Affil ERROR for"+firstname+" "+lastname)
    return(affil_list)

#### Provider page affil scrape test

In [28]:
test = hospSpecFAP_df.iloc[0:10, :]

In [30]:
test_list = doc_affils(test)
pd.DataFrame(test_list)

200
Added Affil for chidi achilefu
200
Added Affil for joan admirand
200
Added Affil for kathleen berg
200
Added Affil for kathleen berg
200
Added Affil for john biboa
200
Added Affil for adam borowski
200
Added Affil for steven burgard
200
Added Affil for larry cartmell
200
Added Affil for larry cartmell
200
Added Affil for cary chisholm


,firstName,hospitalAffiliations,lastName
0,chidi,[Providence Health Center],achilefu
1,joan,[Providence Health Center],admirand
2,kathleen,[Providence Health Center],berg
3,kathleen,[Providence Health Center],berg
4,john,[Providence Health Center],biboa
5,adam,[Providence Health Center],borowski
6,steven,[Providence Health Center],burgard
7,larry,[Providence Health Center],cartmell
8,larry,[Providence Health Center],cartmell
9,cary,[Providence Health Center],chisholm


### Get that data!

In [31]:
prov_affil_list = doc_affils(hospSpecFAP_df)

200
Added Affil for chidi achilefu
200
Added Affil for joan admirand
200
Added Affil for kathleen berg
200
Added Affil for kathleen berg
200
Added Affil for john biboa
200
Added Affil for adam borowski
200
Added Affil for steven burgard
200
Added Affil for larry cartmell
200
Added Affil for larry cartmell
200
Added Affil for cary chisholm
200
Added Affil for g clark
200
Added Affil for amy coffey
200
Added Affil for holly davis
200
Added Affil for larry davis
200
Added Affil for nelson destaffany
200
Added Affil for tamara dutter
200
Added Affil for steven elwell
200
Added Affil for gary fischer
200
Added Affil for nathan forrest
200
Added Affil for paul gaudin
200
Added Affil for gary geldmeier
200
Added Affil for gary geldmeier
200
Added Affil for james goodsett
200
Added Affil for christopher greener
200
Added Affil for rebecca guess
200
Added Affil for casey hector
200
Added Affil for r hendricks
200
Added Affil for eric higginbotham
200
Added Affil for eric higginbotham
200
Added 

In [33]:
#convert to pandas df
Affils_df = pd.DataFrame(prov_affil_list)

In [34]:
#join scraped affils back to roster
FAP_with_affils = pd.merge(hospSpecFAP_df, Affils_df,
              left_on=['FirstNameID', 'LastNameID'],
              right_on=['firstName', 'lastName'],
              how='left')

In [39]:
#check structure
FAP_with_affils.columns

Index(['AddressFormatted', 'Associations', 'Credentials', 'DisplayName',
       'Distance', 'FirstName', 'FirstNameID', 'Groups', 'LastName',
       'LastNameID', 'Latitude', 'Longitude', 'MapAddress', 'PhoneFormatted',
       'Specialties', 'firstName', 'hospitalAffiliations', 'lastName'],
      dtype='object')

In [40]:
FAP_with_affils

,AddressFormatted,Associations,Credentials,DisplayName,Distance,FirstName,FirstNameID,Groups,LastName,LastNameID,Latitude,Longitude,MapAddress,PhoneFormatted,Specialties,firstName,hospitalAffiliations,lastName
0,"405 Londonderry Dr <br />Suite 105 <br />Waco,...",C4T,MD,Chidi Achilefu,-1,Chidi,chidi,"Mid Tex Anesthesia Associates, P.A.",Achilefu,achilefu,31.513468,-9.719951e+01,"405 Londonderry Dr Suite 105 Waco, TX 76712",254-776-0266,Anesthesiology,chidi,[Providence Health Center],achilefu
1,3445 Executive Center Dr. <br />Suite 250 <br ...,,MD,Joan Admirand,-1,Joan,joan,Clinical Pathology Associates,Admirand,admirand,30.359690,-9.774781e+01,"3445 Executive Center Dr. Suite 250 Austin, TX...",512-579-4000,Pathology,joan,[Providence Health Center],admirand
2,"6901 Medical Parkway <br />Waco, TX 76712",,MD,Kathleen Berg,-1,Kathleen,kathleen,"US Acute Care Solutions, US Acute Care Solutio...",Berg,berg,31.514454,-9.719949e+01,"6901 Medical Parkway Waco, TX 76712",254-751-4180,"Pediatric Emergency Medicine, Emergency Medicine",kathleen,[Providence Health Center],berg
3,"6901 Medical Parkway <br />Waco, TX 76712",,MD,Kathleen Berg,-1,Kathleen,kathleen,"US Acute Care Solutions, US Acute Care Solutio...",Berg,berg,31.514454,-9.719949e+01,"6901 Medical Parkway Waco, TX 76712",254-751-4180,"Pediatric Emergency Medicine, Emergency Medicine",kathleen,[Providence Health Center],berg
4,6300 La Calma Dr. <br />Suite 200 <br />Austin...,,MD,Kathleen Berg,-1,Kathleen,kathleen,"US Acute Care Solutions, US Acute Care Solutio...",Berg,berg,30.324444,-9.770303e+01,"6300 La Calma Dr. Suite 200 Austin, TX 78752",512-452-8533,"Pediatric Emergency Medicine, Emergency Medicine",kathleen,[Providence Health Center],berg
5,6300 La Calma Dr. <br />Suite 200 <br />Austin...,,MD,Kathleen Berg,-1,Kathleen,kathleen,"US Acute Care Solutions, US Acute Care Solutio...",Berg,berg,30.324444,-9.770303e+01,"6300 La Calma Dr. Suite 200 Austin, TX 78752",512-452-8533,"Pediatric Emergency Medicine, Emergency Medicine",kathleen,[Providence Health Center],berg
6,"6901 Medical Parkway <br />Waco, TX 76712",C4T,DO,John Biboa,-1,John,john,"McLennan County Hospitalist Services, P.C.",Biboa,biboa,31.514465,-9.719953e+01,"6901 Medical Parkway Waco, TX 76712",254-751-4551,Hospitalist,john,[Providence Health Center],biboa
7,"6901 Medical Parkway <br />Waco, TX 76712",C4T,MD,Adam Borowski,-1,Adam,adam,"Waco Radiology, P.A.",Borowski,borowski,31.514418,-9.719958e+01,"6901 Medical Parkway Waco, TX 76712",254-751-4299,"Radiology, Neuroradiology",adam,[Providence Health Center],borowski
8,"405 Londonderry Dr <br />Suite 105 <br />Waco,...",C4T,MD,Steven Burgard,-1,Steven,steven,"Mid Tex Anesthesia Associates, P.A.",Burgard,burgard,31.513401,-9.719957e+01,"405 Londonderry Dr Suite 105 Waco, TX 76712",254-776-0266,Anesthesiology,steven,[Providence Health Center],burgard
9,601 West State Highway 6 <br />Suite 111 <br /...,C4T,MD,Larry Cartmell,-1,Larry,larry,"Central Texas Pathology Laboratory, P.A.",Cartmell,cartmell,-0.000039,-1.739128e-05,"601 West State Highway 6 Suite 111 Waco, TX 76...",254-752-9621,Pathology,larry,[Providence Health Center],cartmell


#### Initial data carpentry/cleansing

In [41]:
FAP_with_affils.drop_duplicates(subset = ['FirstName', 'LastName', 'MapAddress'], inplace = True)
FAP_with_affils.head()

,AddressFormatted,Associations,Credentials,DisplayName,Distance,FirstName,FirstNameID,Groups,LastName,LastNameID,Latitude,Longitude,MapAddress,PhoneFormatted,Specialties,firstName,hospitalAffiliations,lastName
0,"405 Londonderry Dr <br />Suite 105 <br />Waco,...",C4T,MD,Chidi Achilefu,-1,Chidi,chidi,"Mid Tex Anesthesia Associates, P.A.",Achilefu,achilefu,31.513468,-97.199510,"405 Londonderry Dr Suite 105 Waco, TX 76712",254-776-0266,Anesthesiology,chidi,[Providence Health Center],achilefu
1,3445 Executive Center Dr. <br />Suite 250 <br ...,,MD,Joan Admirand,-1,Joan,joan,Clinical Pathology Associates,Admirand,admirand,30.359690,-97.747810,"3445 Executive Center Dr. Suite 250 Austin, TX...",512-579-4000,Pathology,joan,[Providence Health Center],admirand
2,"6901 Medical Parkway <br />Waco, TX 76712",,MD,Kathleen Berg,-1,Kathleen,kathleen,"US Acute Care Solutions, US Acute Care Solutio...",Berg,berg,31.514454,-97.199490,"6901 Medical Parkway Waco, TX 76712",254-751-4180,"Pediatric Emergency Medicine, Emergency Medicine",kathleen,[Providence Health Center],berg
4,6300 La Calma Dr. <br />Suite 200 <br />Austin...,,MD,Kathleen Berg,-1,Kathleen,kathleen,"US Acute Care Solutions, US Acute Care Solutio...",Berg,berg,30.324444,-97.703026,"6300 La Calma Dr. Suite 200 Austin, TX 78752",512-452-8533,"Pediatric Emergency Medicine, Emergency Medicine",kathleen,[Providence Health Center],berg
6,"6901 Medical Parkway <br />Waco, TX 76712",C4T,DO,John Biboa,-1,John,john,"McLennan County Hospitalist Services, P.C.",Biboa,biboa,31.514465,-97.199530,"6901 Medical Parkway Waco, TX 76712",254-751-4551,Hospitalist,john,[Providence Health Center],biboa


In [65]:
#extract Zip for NPI search
FAP_with_affils['Zip'] = FAP_with_affils.MapAddress.str.extract('(\d{5})')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [66]:
FAP_with_affils_formatted = FAP_with_affils[['FirstName', 'LastName', 'Credentials', 'Specialties', 'Groups', 'MapAddress', 'Zip', 'hospitalAffiliations']]
FAP_with_affils_formatted.head()

,FirstName,LastName,Credentials,Specialties,Groups,MapAddress,Zip,hospitalAffiliations
0,Chidi,Achilefu,MD,Anesthesiology,"Mid Tex Anesthesia Associates, P.A.","405 Londonderry Dr Suite 105 Waco, TX 76712",76712,[Providence Health Center]
1,Joan,Admirand,MD,Pathology,Clinical Pathology Associates,"3445 Executive Center Dr. Suite 250 Austin, TX...",78731,[Providence Health Center]
2,Kathleen,Berg,MD,"Pediatric Emergency Medicine, Emergency Medicine","US Acute Care Solutions, US Acute Care Solutio...","6901 Medical Parkway Waco, TX 76712",76712,[Providence Health Center]
4,Kathleen,Berg,MD,"Pediatric Emergency Medicine, Emergency Medicine","US Acute Care Solutions, US Acute Care Solutio...","6300 La Calma Dr. Suite 200 Austin, TX 78752",78752,[Providence Health Center]
6,John,Biboa,DO,Hospitalist,"McLennan County Hospitalist Services, P.C.","6901 Medical Parkway Waco, TX 76712",76712,[Providence Health Center]


#### Write out Roster and Affils

In [68]:
from pandas import ExcelWriter
filename = 'Providence_HospSpecs_affiliations_20190815.xlsx'

with ExcelWriter(filename) as writer:
        FAP_with_affils_formatted.to_excel(writer, sheet_name='Providence Affiliations')


### STEP 3: Get Accepting new patients from bio pages

For Provider data Quality team--they'd like info for any docs accepting new patients.

In [25]:
fap_no_nans = fap_df.dropna(subset=['FirstNameID', 'LastNameID'])
fap_no_nans.shape

(3381, 15)

In [23]:
def doc_acceptFlags(dataframe):
    accept_list = []
    seconds = 2 #define seconds to wait between get requests
    for index, row in dataframe.iterrows():
        firstname = row['FirstNameID']
        lastname = row['LastNameID']
        try:
            url = "https://www.seton.net/providers/bio/" + firstname + "-" + lastname #concat url
            docpage = requests.get(url,time.sleep(seconds)) #get html from url, with delay to prevent website blacklisting
            print(docpage.status_code)
            soup = BeautifulSoup(docpage.content, 'html5lib') #parse html with BeautifulSoup
            affiliations = soup.find(id="physicianAffiliations") #find physicianAffiliations section of website
            accept_new = soup.find(id="physicianNewPatients").get_text() #get all accept code flags via physicianNewPatients ids
            dict1 = {'firstName': firstname, 'lastName': lastname, 'AcceptFlag': accept_new} #make name and acceptFlags into dictionary
            accept_list.append(dict1)
            print("Added AcceptFlag for "+firstname+" "+lastname)
        except:
            pass
            print("No AcceptFlag for "+firstname+" "+lastname)
    return(accept_list)

In [26]:
AcceptFlags = doc_acceptFlags(fap_no_nans)
print('All DocPages scraped!')

AcceptFlag_df = pd.DataFrame(AcceptFlags)

FAP_with_AcceptFlags = pd.merge(fap_df, AcceptFlag_df,
              left_on=['FirstNameID', 'LastNameID'],
              right_on=['firstName', 'lastName'],
              how='left')

from pandas import ExcelWriter
filename = 'Seton_FAP_AcceptingNewPatients_20190716.xlsx'

with ExcelWriter(filename) as writer:
        FAP_with_AcceptFlags.to_excel(writer, sheet_name='Seton FAP Info')

print('You Excel File is ready!')

200
No AcceptFlag for justin aaker
200
No AcceptFlag for alan abando
200
No AcceptFlag for madhava abburi
200
No AcceptFlag for waleed abdelhafez
200
No AcceptFlag for zainab abdulla
200
No AcceptFlag for zainab abdulla
200
No AcceptFlag for arthur abello
200
No AcceptFlag for william abide
200
No AcceptFlag for john abikhaled
200
No AcceptFlag for shannon abikhaled
200
No AcceptFlag for sami aboumatar
200
No AcceptFlag for john abraham
200
No AcceptFlag for merlin abraham
200
No AcceptFlag for joshua abramowitz
200
No AcceptFlag for david abrams
200
No AcceptFlag for steven abrams
200
No AcceptFlag for eric acheson
200
No AcceptFlag for anne adams
200
No AcceptFlag for bradley adams
200
No AcceptFlag for clayton adams
200
No AcceptFlag for david adams
200
Added AcceptFlag for adewole adamson
200
Added AcceptFlag for adewole adamson
200
Added AcceptFlag for adewole adamson
200
No AcceptFlag for debaroti addy
200
No AcceptFlag for eric adelman
200
No AcceptFlag for sikander adeni
200
No

200
No AcceptFlag for matthew benevich
200
No AcceptFlag for hans bengtson
200
No AcceptFlag for thomas bening
200
No AcceptFlag for bari bennett
200
No AcceptFlag for dwayne bennett
200
Added AcceptFlag for terrell benold
200
No AcceptFlag for edward benton
200
No AcceptFlag for jacques benun
200
No AcceptFlag for craig berent
200
No AcceptFlag for kathleen berg
200
No AcceptFlag for kathleen berg
200
No AcceptFlag for lloyd berg
200
No AcceptFlag for brian berger
200
No AcceptFlag for brian berger
200
No AcceptFlag for dennis berger
200
No AcceptFlag for michelle berger
200
Added AcceptFlag for ryan bergeson
200
Added AcceptFlag for deborah bergfeld
200
Added AcceptFlag for deborah bergfeld
200
Added AcceptFlag for rahel berhane
200
No AcceptFlag for richard berkowitz
200
No AcceptFlag for nicole bernard
200
No AcceptFlag for kurt berneburg
200
No AcceptFlag for david berry
200
No AcceptFlag for crystal berry_roberts
200
No AcceptFlag for john bertelson
200
No AcceptFlag for john ber

200
Added AcceptFlag for david carmack
200
No AcceptFlag for daniel carrasco
200
No AcceptFlag for benito carrera_leal
200
No AcceptFlag for erica carroll
200
Added AcceptFlag for tannon carroll
200
Added AcceptFlag for tannon carroll
200
Added AcceptFlag for tannon carroll
200
Added AcceptFlag for tannon carroll
200
No AcceptFlag for jack carsner
200
Added AcceptFlag for kimberly carter
200
No AcceptFlag for shelby carter
200
No AcceptFlag for joseph casadonte
200
No AcceptFlag for luis casaubon
200
No AcceptFlag for ann_marie case
200
Added AcceptFlag for julia case
200
No AcceptFlag for kelly casey
200
No AcceptFlag for ryan casey
200
Added AcceptFlag for harry casmedes
200
No AcceptFlag for melody cassels
200
No AcceptFlag for cathy castillo
200
No AcceptFlag for marcela castillo
200
No AcceptFlag for rene castillo
200
No AcceptFlag for gregory catlett
200
Added AcceptFlag for clay cauthen
200
Added AcceptFlag for clay cauthen
200
No AcceptFlag for paul cauvin
200
No AcceptFlag for

200
No AcceptFlag for christopher danney
200
No AcceptFlag for kim dao
200
Added AcceptFlag for yen dao
200
No AcceptFlag for byron darby
200
No AcceptFlag for adrienne darhower
200
No AcceptFlag for christopher darnall
200
No AcceptFlag for ashima das
200
No AcceptFlag for rani das
200
No AcceptFlag for nilanjana dasgupta
200
No AcceptFlag for swati date
200
No AcceptFlag for meredith davenport
200
No AcceptFlag for elbert david
200
No AcceptFlag for antonia davidson
200
No AcceptFlag for joel davidson
200
Added AcceptFlag for juan davila
200
No AcceptFlag for jenna davis
200
No AcceptFlag for joanne davis
200
No AcceptFlag for l_l_tad davis
200
No AcceptFlag for lindsay davis
200
No AcceptFlag for lindsey davis
200
No AcceptFlag for holly davison
200
No AcceptFlag for mark dawson
200
No AcceptFlag for thomas dawson
200
No AcceptFlag for avis day
200
No AcceptFlag for barakah day
200
No AcceptFlag for michael de_lota
200
No AcceptFlag for brendan de_marco
200
No AcceptFlag for ximena 

200
Added AcceptFlag for sarah felderhoff
200
No AcceptFlag for david feldman
200
No AcceptFlag for mark felger
200
No AcceptFlag for arnold fenrich
200
Added AcceptFlag for amber fenton
200
No AcceptFlag for david ferguson
200
No AcceptFlag for jim fernandez
200
Added AcceptFlag for marisol fernandez
200
Added AcceptFlag for marisol fernandez
200
No AcceptFlag for sarah ferrero
200
Added AcceptFlag for james ferriss
200
Added AcceptFlag for james ferriss
200
No AcceptFlag for jason feuerman
200
Added AcceptFlag for w_ fielder
200
No AcceptFlag for esther fields
200
No AcceptFlag for jon filardi
200
No AcceptFlag for eric fillman
200
No AcceptFlag for gerald fincken
200
No AcceptFlag for julie fisher
200
No AcceptFlag for rebecca fisher
200
No AcceptFlag for avrim fishkind
200
No AcceptFlag for t fitzpatrick
200
No AcceptFlag for david fleeger
200
No AcceptFlag for lauren fleischer
200
No AcceptFlag for richard fleming
200
No AcceptFlag for jeanne flinn
200
No AcceptFlag for derrick fl

...

200
No AcceptFlag for khris ramdeen
200
No AcceptFlag for carmen ramirez
200
No AcceptFlag for jaime ramirez
200
No AcceptFlag for raul ramirez
200
No AcceptFlag for louis ramos
200
No AcceptFlag for louis ramos
200
Added AcceptFlag for ankur rana
200
Added AcceptFlag for ankur rana
200
No AcceptFlag for nehal rana
200
No AcceptFlag for jennifer ranario
200
No AcceptFlag for poonam rane
200
Added AcceptFlag for anuradha rangarajan
200
Added AcceptFlag for sujal rangwalla
200
Added AcceptFlag for sujal rangwalla
200
Added AcceptFlag for sujal rangwalla
200
No AcceptFlag for fara ranjbaran
200
No AcceptFlag for murali ranjithan
200
No AcceptFlag for marcia rannefeld
200
No AcceptFlag for sandeep rao
200
No AcceptFlag for emily rasco
200
No AcceptFlag for alan rashid
200
No AcceptFlag for jay rasmussen
200
No AcceptFlag for daniel ratcliff
200
Added AcceptFlag for sheri ravenscroft
200
Added AcceptFlag for sheri ravenscroft
200
No AcceptFlag for vandana rawal
200
No AcceptFlag for sterlin

200
No AcceptFlag for j scharnberg
200
No AcceptFlag for david schauer
200
No AcceptFlag for michael schindel
200
No AcceptFlag for jeffrey schlab
200
No AcceptFlag for anna schlechter
200
Added AcceptFlag for robert schlechter
200
Added AcceptFlag for robert schlechter
200
No AcceptFlag for john schlitt
200
No AcceptFlag for rodney schmidt
200
No AcceptFlag for martha schmitz
200
No AcceptFlag for matthew schmitz
200
No AcceptFlag for adam schneider
200
No AcceptFlag for douglas schneider
200
No AcceptFlag for lisa schneider
200
No AcceptFlag for christopher schneller
200
No AcceptFlag for eugene schoch
200
No AcceptFlag for sandford schocket
200
No AcceptFlag for peter scholl
200
No AcceptFlag for peter scholl
200
No AcceptFlag for richard schram
200
No AcceptFlag for jana schrier
200
No AcceptFlag for w schultz
200
No AcceptFlag for jeannine schuman_mccoy
200
No AcceptFlag for brian schwab
200
No AcceptFlag for joanna schwartz
200
No AcceptFlag for william schwartz
200
No AcceptFlag

200
No AcceptFlag for perry stevens
200
No AcceptFlag for daniel stewart
200
No AcceptFlag for eileen stewart
200
No AcceptFlag for mark stewart
200
No AcceptFlag for robert stewart
200
No AcceptFlag for ashley stiegler
200
No AcceptFlag for karen stierman
200
No AcceptFlag for kathleen stinson
200
No AcceptFlag for kristopher stockton
200
Added AcceptFlag for john stokes
200
No AcceptFlag for lanny stone
200
No AcceptFlag for richard stovall
200
No AcceptFlag for jenna strahan
200
No AcceptFlag for stephen strakowski
200
No AcceptFlag for daniel stromberg
200
No AcceptFlag for robert stroud
200
No AcceptFlag for sandra stuetelberg
200
No AcceptFlag for paige suffredini
200
No AcceptFlag for aditi sule
200
No AcceptFlag for brian sullivan
200
No AcceptFlag for jennifer summers
200
No AcceptFlag for krishna surapaneni
200
No AcceptFlag for veena surapaneni
200
No AcceptFlag for vikram suraparaju
200
No AcceptFlag for vikram suraparaju
200
No AcceptFlag for zachary sussman
200
No AcceptF

200
No AcceptFlag for aleksandra vidacic
200
No AcceptFlag for jay viernes
200
No AcceptFlag for paul vigo
200
No AcceptFlag for mike vik
200
No AcceptFlag for john villacis
200
No AcceptFlag for arturo villarreal
200
Added AcceptFlag for ross vines
200
No AcceptFlag for sanjay vinjamaram
200
No AcceptFlag for malavika vinta
200
No AcceptFlag for amar vira
200
No AcceptFlag for anisha virani
200
No AcceptFlag for anisha virani
200
No AcceptFlag for jaclyn virant
200
No AcceptFlag for david vo
200
No AcceptFlag for glen vo
200
No AcceptFlag for phu vo
200
No AcceptFlag for laurier vocal
200
Added AcceptFlag for katie volkers
200
No AcceptFlag for joseph volpe
200
No AcceptFlag for fred voorhees
200
No AcceptFlag for nandagopal vrindavanam
200
No AcceptFlag for anna vu_wallace
200
No AcceptFlag for julie wachtel
200
No AcceptFlag for julie wachtel
200
No AcceptFlag for galen wachtman
200
No AcceptFlag for john wages
200
No AcceptFlag for nicholas wagner
200
Added AcceptFlag for tara wagn

200
No AcceptFlag for michael yium
200
No AcceptFlag for vanessa yium
200
No AcceptFlag for prameela yoganandan
200
No AcceptFlag for sonja yoo
200
No AcceptFlag for jeffrey yorio
200
No AcceptFlag for jennifer york
200
No AcceptFlag for kathy yoshimura
200
No AcceptFlag for joseph youman
200
No AcceptFlag for joseph youman
200
No AcceptFlag for amy young
200
No AcceptFlag for cindy_ young
200
No AcceptFlag for lindsay young
200
No AcceptFlag for stanford young
200
No AcceptFlag for farheen yousuf
200
No AcceptFlag for sonia yousuf
200
No AcceptFlag for olivia yun
200
No AcceptFlag for stephen yurco
200
Added AcceptFlag for ajay zachariah
200
No AcceptFlag for aveen zachariah
200
No AcceptFlag for james zachary
200
Added AcceptFlag for jamie zachrison
200
Added AcceptFlag for jamie zachrison
200
No AcceptFlag for jason zagrodzky
200
No AcceptFlag for jason zagrodzky
200
No AcceptFlag for jason zagrodzky
200
No AcceptFlag for nida zakiullah
200
No AcceptFlag for nida zakiullah
200
No Ac

### STEP 4: Get Prac NPIs for crossreference

In order to cross reference the docs against our current db, we'll need their individual National Practitioner Number (NPI). For individual docs, thats a Type 1 NPI. We can get that using the NPPES API.


CAVEAT: The NPPES API is not foolproof. We'll use name and zipcode to search for NPI, but if there are multiple docs with the same name working in the same Zip, we'll get multiple results.  Here, I'll capture specialty as well, to check against the specialty listed on the hosp roster.  There will still need to be some manual curation of ambiguous NPI matches.  Also, there may need to be manual NPI searches for docs with no NPI results--sometimes they will be listed under previous names or addresses. These edge cases are too specific for a current programatic scrape.

In [71]:
# Put it all together!
import pandas as pd
import json
import time
import requests
from bs4 import BeautifulSoup
import re
from pandas.io.json import json_normalize

def NPI_API_PROVIDER_NAME(dataframe, delay):
    NPI_API = pd.DataFrame(columns=['basic.first_name', 'basic.last_name','number',  'address_1', 'address_2', 'address_purpose', 'city', 'state', 'postal_code', 'telephone_number', 'fax_number', 'basic.enumeration_date']) #initialize DataFrame
    seconds = delay #define seconds to wait between get requests
    
    for index, row in dataframe.iterrows():
        url_1 = 'https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&first_name='
        url_first_name = row['FirstName']
        url_2 = '&use_first_name_alias=&last_name='
        url_last_name = row['LastName']
        url_3 = '&organization_name=&address_purpose=LOCATION&city=&state=TX&postal_code='
        url_zip = row['Zip']
        url_4 = '&country_code=&limit=&skip=&version=2.1'
        url = url_1+url_first_name+url_2+url_last_name+url_3+url_zip+url_4
        print('Query url: '+ url)
        try:
            page = requests.get(url, time.sleep(seconds))
            #print(page.status_code) #for testing
            page_json = json.loads(page.content)
            df = pd.DataFrame.from_dict(json_normalize(page_json['results']), orient='columns')
            taxonomies_with_NPI = pd.io.json.json_normalize(page_json['results'], record_path='taxonomies', meta=['number'])
            df_addresses = pd.merge(df, taxonomies_with_NPI, how='inner', on='number')
            columns_to_keep = ['basic.first_name', 'basic.last_name', 'number',  'desc']
            df_taxonomies_subset = df_addresses.reindex(columns=columns_to_keep)
            #print(df_addresses_subset.head(1)) #for testing
            NPI_API = NPI_API.append(df_taxonomies_subset, ignore_index=True)
            #print(NPI_API.shape) #for testing
            print('NPI added for '+ url_first_name+' '+url_last_name)
        except:
            pass
            print('NPI ERROR for '+ url_first_name+' '+url_last_name)
    return(NPI_API)

In [72]:
providence_prac_npi = NPI_API_PROVIDER_NAME(FAP_with_affils_formatted, 2)

Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&first_name=Chidi&use_first_name_alias=&last_name=Achilefu&organization_name=&address_purpose=LOCATION&city=&state=TX&postal_code=76712&country_code=&limit=&skip=&version=2.1
NPI ERROR for Chidi Achilefu
Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&first_name=Joan&use_first_name_alias=&last_name=Admirand&organization_name=&address_purpose=LOCATION&city=&state=TX&postal_code=78731&country_code=&limit=&skip=&version=2.1
NPI added for Joan Admirand
Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&first_name=Kathleen&use_first_name_alias=&last_name=Berg&organization_name=&address_purpose=LOCATION&city=&state=TX&postal_code=76712&country_code=&limit=&skip=&version=2.1
NPI ERROR for Kathleen Berg
Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&first_name=

NPI ERROR for Eric Higginbotham
Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&first_name=Eric&use_first_name_alias=&last_name=Higginbotham&organization_name=&address_purpose=LOCATION&city=&state=TX&postal_code=76712&country_code=&limit=&skip=&version=2.1
NPI ERROR for Eric Higginbotham
Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&first_name=Adam&use_first_name_alias=&last_name=Johnson&organization_name=&address_purpose=LOCATION&city=&state=TX&postal_code=76710&country_code=&limit=&skip=&version=2.1
NPI ERROR for Adam Johnson
Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&first_name=Adam&use_first_name_alias=&last_name=Johnson&organization_name=&address_purpose=LOCATION&city=&state=TX&postal_code=76712&country_code=&limit=&skip=&version=2.1
NPI added for Adam Johnson
Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=

NPI added for Claudius Robinson
Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&first_name=Roxann&use_first_name_alias=&last_name=Samples&organization_name=&address_purpose=LOCATION&city=&state=TX&postal_code=76712&country_code=&limit=&skip=&version=2.1
NPI added for Roxann Samples
Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&first_name=Roxann&use_first_name_alias=&last_name=Samples&organization_name=&address_purpose=LOCATION&city=&state=TX&postal_code=76712&country_code=&limit=&skip=&version=2.1
NPI added for Roxann Samples
Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&first_name=Timothy&use_first_name_alias=&last_name=Schmidt&organization_name=&address_purpose=LOCATION&city=&state=TX&postal_code=76712&country_code=&limit=&skip=&version=2.1
NPI added for Timothy Schmidt
Query url: https://npiregistry.cms.hhs.gov/api/?number=&enumeration_t

#### NPI data carpentry

In [75]:
#providence_prac_npiLocations = providence_prac_npi[providence_prac_npi["address_purpose"]=='LOCATION']
providence_prac_npi

,address_1,address_2,address_purpose,basic.enumeration_date,basic.first_name,basic.last_name,city,desc,fax_number,number,postal_code,state,telephone_number
0,NaN,NaN,NaN,NaN,JOAN,ADMIRAND,NaN,Pathology Anatomic Pathology,NaN,1356441596,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,JOHN,BIBOA,NaN,Internal Medicine,NaN,1083990584,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,STEVEN,BURGARD,NaN,Anesthesiology,NaN,1851480438,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,AMY,COFFEY,NaN,Student in an Organized Health Care Education/...,NaN,1417223389,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,AMY,COFFEY,NaN,Pathology Anatomic Pathology,NaN,1417223389,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,HOLLY,DAVIS,NaN,Anesthesiology,NaN,1225127749,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,LARRY,DAVIS,NaN,Legal Medicine,NaN,1285600668,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NELSON,DESTAFFANY,NaN,Anesthesiology,NaN,1376598912,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,STEVEN,ELWELL,NaN,Internal Medicine,NaN,1336112556,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,GARY,FISCHER,NaN,Anesthesiology,NaN,1578574299,NaN,NaN,NaN


In [77]:
providence_prac_npiLocations_npi_only = providence_prac_npi.loc[:,['basic.first_name', 'basic.last_name', 'number', 'desc']]
providence_prac_npiLocations_npi_only.drop_duplicates(inplace = True)
providence_prac_npiLocations_npi_only.head()

,basic.first_name,basic.last_name,number,desc
0,JOAN,ADMIRAND,1356441596,Pathology Anatomic Pathology
1,JOHN,BIBOA,1083990584,Internal Medicine
2,STEVEN,BURGARD,1851480438,Anesthesiology
3,AMY,COFFEY,1417223389,Student in an Organized Health Care Education/...
4,AMY,COFFEY,1417223389,Pathology Anatomic Pathology


In [78]:
providence_prac_npiLocations_npi_only['FirstName'] = providence_prac_npiLocations_npi_only['basic.first_name'].str.title()
providence_prac_npiLocations_npi_only['LastName'] = providence_prac_npiLocations_npi_only['basic.last_name'].str.title()

providence_hospital_affils_npi = pd.merge(FAP_with_affils_formatted, providence_prac_npiLocations_npi_only, 
                               how = 'left',
                               on = ['FirstName', 'LastName'])
providence_hospital_affils_npi.head()

,FirstName,LastName,Credentials,Specialties,Groups,MapAddress,Zip,hospitalAffiliations,basic.first_name,basic.last_name,number,desc
0,Chidi,Achilefu,MD,Anesthesiology,"Mid Tex Anesthesia Associates, P.A.","405 Londonderry Dr Suite 105 Waco, TX 76712",76712,[Providence Health Center],NaN,NaN,NaN,NaN
1,Joan,Admirand,MD,Pathology,Clinical Pathology Associates,"3445 Executive Center Dr. Suite 250 Austin, TX...",78731,[Providence Health Center],JOAN,ADMIRAND,1356441596,Pathology Anatomic Pathology
2,Kathleen,Berg,MD,"Pediatric Emergency Medicine, Emergency Medicine","US Acute Care Solutions, US Acute Care Solutio...","6901 Medical Parkway Waco, TX 76712",76712,[Providence Health Center],NaN,NaN,NaN,NaN
3,Kathleen,Berg,MD,"Pediatric Emergency Medicine, Emergency Medicine","US Acute Care Solutions, US Acute Care Solutio...","6300 La Calma Dr. Suite 200 Austin, TX 78752",78752,[Providence Health Center],NaN,NaN,NaN,NaN
4,John,Biboa,DO,Hospitalist,"McLennan County Hospitalist Services, P.C.","6901 Medical Parkway Waco, TX 76712",76712,[Providence Health Center],JOHN,BIBOA,1083990584,Internal Medicine


In [59]:
providence_hospital_affils_npi.columns

Index(['FirstName', 'LastName', 'Credentials', 'Specialties', 'Groups',
       'MapAddress', 'hospitalAffiliations', 'basic.first_name',
       'basic.last_name', 'number'],
      dtype='object')

In [79]:
providence_subset = providence_hospital_affils_npi[['FirstName', 'LastName', 'number', 'Credentials', 'Specialties', 'desc','Groups',
       'MapAddress', 'hospitalAffiliations']]
providence_subset.head()

,FirstName,LastName,number,Credentials,Specialties,desc,Groups,MapAddress,hospitalAffiliations
0,Chidi,Achilefu,NaN,MD,Anesthesiology,NaN,"Mid Tex Anesthesia Associates, P.A.","405 Londonderry Dr Suite 105 Waco, TX 76712",[Providence Health Center]
1,Joan,Admirand,1356441596,MD,Pathology,Pathology Anatomic Pathology,Clinical Pathology Associates,"3445 Executive Center Dr. Suite 250 Austin, TX...",[Providence Health Center]
2,Kathleen,Berg,NaN,MD,"Pediatric Emergency Medicine, Emergency Medicine",NaN,"US Acute Care Solutions, US Acute Care Solutio...","6901 Medical Parkway Waco, TX 76712",[Providence Health Center]
3,Kathleen,Berg,NaN,MD,"Pediatric Emergency Medicine, Emergency Medicine",NaN,"US Acute Care Solutions, US Acute Care Solutio...","6300 La Calma Dr. Suite 200 Austin, TX 78752",[Providence Health Center]
4,John,Biboa,1083990584,DO,Hospitalist,Internal Medicine,"McLennan County Hospitalist Services, P.C.","6901 Medical Parkway Waco, TX 76712",[Providence Health Center]


In [80]:
from pandas import ExcelWriter
filename = 'Providence_HospSpecialties_Affiliations_with_NPI.xlsx'

with ExcelWriter(filename) as writer:
        providence_subset.to_excel(writer, sheet_name='ProvidenceHospSpecAffils')